In [1]:
import requests
import json
import pandas as pd
import sqlalchemy as sa
import time

In [2]:
appid = '8782e17b1bcc828a0a1ac94b627f05be'
lat = 43.6548
lon = -79.3883

api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}'.format(lat,lon,appid)

print(api_url)

https://api.openweathermap.org/data/2.5/weather?lat=43.6548&lon=-79.3883&appid=8782e17b1bcc828a0a1ac94b627f05be


In [3]:
api_response = requests.get(api_url)
api_response

<Response [200]>

In [4]:
data = api_response.json()
data

{'coord': {'lon': -79.3927, 'lat': 43.6514},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 298.09,
  'feels_like': 298.18,
  'temp_min': 295.91,
  'temp_max': 299.7,
  'pressure': 1006,
  'humidity': 59},
 'visibility': 10000,
 'wind': {'speed': 7.72, 'deg': 320, 'gust': 10.29},
 'clouds': {'all': 0},
 'dt': 1691523332,
 'sys': {'type': 1,
  'id': 718,
  'country': 'CA',
  'sunrise': 1691489644,
  'sunset': 1691541148},
 'timezone': -14400,
 'id': 6167863,
 'name': 'Downtown Toronto',
 'cod': 200}

In [5]:
data['weather']

[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]

In [6]:
weather_data = pd.DataFrame(data['weather'])
weather_data.head(10)

,id,main,description,icon
0,800,Clear,clear sky,01d


### Data Cleaning

In [7]:
weather_data['id'].fillna(0,inplace = True)
weather_data['main'].fillna('others',inplace = True)
weather_data['description'].fillna('others',inplace = True)

### Data Engineering

In [8]:
relevant_data = ['id','main','description']
df = weather_data[relevant_data]

In [9]:
db_secret = {
    'drivername' : 'postgresql+psycopg2',
    'host'       : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'       : '5432',
    'username'   : 'hunterf',
    'password'   : '2023!Schulich',
    'database'   : 'hunterf_db'
}

In [10]:
db_connection_url = sa.engine.URL.create(
    drivername = db_secret['drivername'],
    username   = db_secret['username'],
    password   = db_secret['password'],
    host       = db_secret['host'],
    port       = db_secret['port'],
    database   = db_secret['database']
)

In [11]:
engine = sa.create_engine(db_connection_url)

In [12]:
engine.execute("CREATE SCHEMA IF NOT EXISTS p2_weather_data;")
df.to_sql(
    name = 'weather_data',
    schema = 'p2_weather_data',
    con = engine,
    if_exists = 'replace',
    index = False,
    method='multi',
    dtype= {
    'id'          : sa.types.DECIMAL(10,0),
    'main'        : sa.types.String,
    'description' : sa.types.String,
    'icon'        : sa.types.String
}
)

In [13]:
appid = '8782e17b1bcc828a0a1ac94b627f05be'
lat = 43.6548
lon = -79.3883
units = 'metric'

api_url_forecast = 'https://api.openweathermap.org/data/2.5/forecast?lat={}&lon={}&appid={}&units'.format(lat,lon,appid,units)

In [14]:
api_response_forecast = requests.get(api_url_forecast)
api_response_forecast

<Response [200]>

In [15]:
data_forecast = api_response_forecast.json()
data_forecast

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1691528400,
   'main': {'temp': 298.09,
    'feels_like': 298.18,
    'temp_min': 297.75,
    'temp_max': 298.09,
    'pressure': 1006,
    'sea_level': 1006,
    'grnd_level': 996,
    'humidity': 59,
    'temp_kf': 0.34},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 5.63, 'deg': 313, 'gust': 10.01},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2023-08-08 21:00:00'},
  {'dt': 1691539200,
   'main': {'temp': 297.32,
    'feels_like': 297.33,
    'temp_min': 295.78,
    'temp_max': 297.32,
    'pressure': 1006,
    'sea_level': 1006,
    'grnd_level': 997,
    'humidity': 59,
    'temp_kf': 1.54},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 33},
   'wind': {'speed': 4.35, 'deg': 298, 'gust': 8.98},
   'visibi

In [16]:
all_main_data = [item['main'] for item in data_forecast['list']]
print(all_main_data)

[{'temp': 298.09, 'feels_like': 298.18, 'temp_min': 297.75, 'temp_max': 298.09, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 996, 'humidity': 59, 'temp_kf': 0.34}, {'temp': 297.32, 'feels_like': 297.33, 'temp_min': 295.78, 'temp_max': 297.32, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 997, 'humidity': 59, 'temp_kf': 1.54}, {'temp': 294.92, 'feels_like': 294.93, 'temp_min': 293.33, 'temp_max': 294.92, 'pressure': 1007, 'sea_level': 1007, 'grnd_level': 998, 'humidity': 68, 'temp_kf': 1.59}, {'temp': 292.28, 'feels_like': 292.1, 'temp_min': 292.28, 'temp_max': 292.28, 'pressure': 1008, 'sea_level': 1008, 'grnd_level': 997, 'humidity': 71, 'temp_kf': 0}, {'temp': 291.65, 'feels_like': 291.46, 'temp_min': 291.65, 'temp_max': 291.65, 'pressure': 1008, 'sea_level': 1008, 'grnd_level': 997, 'humidity': 73, 'temp_kf': 0}, {'temp': 292.53, 'feels_like': 292.43, 'temp_min': 292.53, 'temp_max': 292.53, 'pressure': 1008, 'sea_level': 1008, 'grnd_level': 998, 'humidity': 73, 'temp_kf':

In [17]:
weather_forecast = pd.DataFrame(all_main_data)
weather_forecast.head(10)

,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf
0,298.09,298.18,297.75,298.09,1006,1006,996,59,0.34
1,297.32,297.33,295.78,297.32,1006,1006,997,59,1.54
2,294.92,294.93,293.33,294.92,1007,1007,998,68,1.59
3,292.28,292.10,292.28,292.28,1008,1008,997,71,0.00
4,291.65,291.46,291.65,291.65,1008,1008,997,73,0.00
5,292.53,292.43,292.53,292.53,1008,1008,998,73,0.00
6,297.16,297.10,297.16,297.16,1008,1008,998,57,0.00
7,299.79,299.79,299.79,299.79,1007,1007,997,47,0.00
8,301.00,300.75,301.00,301.00,1006,1006,996,41,0.00
9,297.85,297.89,297.85,297.85,1006,1006,996,58,0.00


### Data Cleaning

In [18]:
weather_forecast['temp'].fillna(0,inplace = True)
weather_forecast['feels_like'].fillna(0,inplace = True)
weather_forecast['pressure'].fillna(0,inplace = True)
weather_forecast['sea_level'].fillna(0,inplace = True)
weather_forecast['grnd_level'].fillna(0,inplace = True)
weather_forecast['humidity'].fillna(0,inplace = True)
weather_forecast['temp_kf'].fillna(0,inplace = True)

### Data Engineering

In [19]:
relevant_columns = ['temp','feels_like','pressure','sea_level','grnd_level','humidity','temp_kf']
df_forecast = weather_forecast[relevant_columns]

In [20]:
engine.execute("CREATE SCHEMA IF NOT EXISTS p2_weather_data;")
df_forecast.to_sql(
    name = 'weather_forecast',
    schema = 'p2_weather_data',
    con = engine,
    if_exists = 'replace',
    index = False,
    method='multi',
    dtype= {
    'temp'            : sa.types.DECIMAL(10,0),
    'feels_like'      : sa.types.DECIMAL(10,0),
    'temp_min'        : sa.types.DECIMAL(10,0),
    'temp_max'        : sa.types.DECIMAL(10,0),
    'pressure'        : sa.types.DECIMAL(10,0),
    'sea_level'       : sa.types.DECIMAL(10,0),
    'grnd_level'      : sa.types.DECIMAL(10,0),
    'humidity'        : sa.types.DECIMAL(10,0),
    'temp_kf'         : sa.types.DECIMAL(10,0)
}
)

In [21]:
import xml.etree.ElementTree as ET

In [22]:
xml_string = """<?xml version='1.0' encoding='UTF-8' standalone='no'?>
<alert xmlns="urn:oasis:names:tc:emergency:cap:1.2">
    <identifier>urn:oid:2.49.0.1.124.1508128505.2023</identifier>
    <sender>cap-pac@canada.ca</sender>
    <sent>2023-08-08T00:04:09-00:00</sent>
    <status>Actual</status>
    <msgType>Update</msgType>
    <source>Env. Can. - Can. Met. Ctr. – Montréal</source>
    <scope>Public</scope>
    <code>profile:CAP-CP:0.4</code>
    <code>layer:SOREM:1.0</code>
    <code>layer:EC-MSC-SMC:1.0</code>
    <code>layer:EC-MSC-SMC:1.1</code>
    <code>layer:SOREM:2.0</code>
    <note>Service Notice - February 2023: The Environment and Climate Change Canada (ECCC) CAP Service undergoes changes from time to time as the business of alerting evolves. For 2023, changes will include... 1) modifications, or even removal, of the Wireless Public Alerting Message (WPAM) layer due to the successful pilot of the UpdateX feature recently deployed in the Canadian Cell Broadcasting Environment, and 2) other minor improvements and corrections. For more information on these changes, you are invited to subscribe to the following mailing list: http://lists.cmc.ec.gc.ca/mailman/listinfo/dd_info | Notification de service - février 2023: Le service du PAC d’Environnement et Changement climatique Canada (ECCC) subit périodiquement des changements et ajouts à mesure que le système d’alerte évolue. Pour 2023, ces changements incluent... 1) des modifications, voire la suppression, de la couche du Service d’alertes sans fil au public (WPAM) en raison du succès du projet pilote de la fonctionnalité UpdateX récemment déployée dans l’environnement de diffusion sans fil canadien, et 2) d’autres améliorations et corrections mineures. Pour plus d’informations sur ces changements, vous êtes invités à vous inscrire à la liste de diffusion suivante: http://lists.cmc.ec.gc.ca/mailman/listinfo/dd_info</note>
    <references>cap-pac@canada.ca,urn:oid:2.49.0.1.124.0681602026.2023,2023-08-07T23:40:03-00:00 cap-pac@canada.ca,urn:oid:2.49.0.1.124.3853882691.2023,2023-08-07T23:41:25-00:00</references>
    <info>
        <language>en-CA</language>
        <category>Met</category>
        <event>thunderstorm</event>
        <responseType>AllClear</responseType>
        <urgency>Past</urgency>
        <severity>Minor</severity>
        <certainty>Observed</certainty>
        <audience>general public</audience>
        <eventCode>
            <valueName>profile:CAP-CP:Event:0.4</valueName>
            <value>thunderstorm</value>
        </eventCode>
        <eventCode>
            <valueName>SAME</valueName>
            <value>SVR</value>
        </eventCode>
        <effective>2023-08-08T00:04:09-00:00</effective>
        <expires>2023-08-08T01:04:09-00:00</expires>
        <senderName>Environment Canada</senderName>
        <headline>severe thunderstorm warning ended</headline>
        <description>
Severe thunderstorms have weakened or moved out of the region.


###
</description>
        <instruction/>
        <web>http://weather.gc.ca/warnings/index_e.html?prov=ab</web>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Alert_Type</valueName>
            <value>warning</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Broadcast_Intrusive</valueName>
            <value>no</value>
        </parameter>
        <parameter>
            <valueName>layer:SOREM:1.0:Broadcast_Immediately</valueName>
            <value>No</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.1:Parent_URI</valueName>
            <value>msc/alert/environment/hazard/alert-3.0-ascii/consolidated-xml-2.0/20230808000409692/AB_00_01_LAND/STW/2340035092045549953202308070501_AB_00_01_LAND/actual/en_proper_complete_c-fr_proper_complete_c/NinJo</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.1:CAP_count</valueName>
            <value>A:7953 M:43700 C:52914</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Alert_Location_Status</valueName>
            <value>ended</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Alert_Name</valueName>
            <value>severe thunderstorm warning</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Alert_Coverage</valueName>
            <value>Alberta</value>
        </parameter>
        <parameter>
            <valueName>layer:SOREM:1.0:Broadcast_Text</valueName>
            <value>At 6:04 p.m. Mountain Daylight Time Monday, Environment Canada has ended a severe thunderstorm warning for Northwestern Alberta.  Cleardale and Hines Creek. Severe thunderstorms have weakened or moved out of the region. </value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.1:Designation_Code</valueName>
            <value>AB_00_01_LAND</value>
        </parameter>
        <parameter>
            <valueName>layer:SOREM:2.0:WirelessImmediate</valueName>
            <value>No</value>
        </parameter>
        <parameter>
            <valueName>layer:SOREM:2.0:WirelessText</valueName>
            <value>At 6:04 p.m. Mountain Daylight Time Monday, Environment Canada has ended a severe thunderstorm warning for this mobile coverage area. </value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.1:Alert_Location_Status</valueName>
            <value>ended</value>
        </parameter>
        <area>
            <areaDesc>Clear Hills Co. near Cleardale Worsley and Cherry Point</areaDesc>
            <polygon>56.2761,-118.9797 56.2173,-119.0834 56.2298,-119.1816 56.179,-119.3963 56.0766,-119.8228 56.0691,-120.0013 56.1476,-120.0013 56.5097,-120.0139 56.7699,-120.0013 56.7769,-119.4783 56.7699,-118.979 56.3497,-118.9796 56.2761,-118.9797</polygon>
            <geocode>
                <valueName>layer:EC-MSC-SMC:1.0:CLC</valueName>
                <value>078212</value>
            </geocode>
            <geocode>
                <valueName>profile:CAP-CP:Location:0.3</valueName>
                <value>4817062</value>
            </geocode>
        </area>
    </info>
    <info>
        <language>fr-CA</language>
        <category>Met</category>
        <event>orages</event>
        <responseType>AllClear</responseType>
        <urgency>Past</urgency>
        <severity>Minor</severity>
        <certainty>Observed</certainty>
        <audience>grand public</audience>
        <eventCode>
            <valueName>profile:CAP-CP:Event:0.4</valueName>
            <value>thunderstorm</value>
        </eventCode>
        <eventCode>
            <valueName>SAME</valueName>
            <value>SVR</value>
        </eventCode>
        <effective>2023-08-08T00:04:09-00:00</effective>
        <expires>2023-08-08T01:04:09-00:00</expires>
        <senderName>Environnement Canada</senderName>
        <headline>alerte d'orages violents annulée</headline>
        <description>
Les orages violents ont faibli ou ont quitté la région.

###
</description>
        <instruction/>
        <web>http://meteo.gc.ca/warnings/index_f.html?prov=ab</web>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Alert_Type</valueName>
            <value>warning</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Broadcast_Intrusive</valueName>
            <value>no</value>
        </parameter>
        <parameter>
            <valueName>layer:SOREM:1.0:Broadcast_Immediately</valueName>
            <value>No</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.1:Parent_URI</valueName>
            <value>msc/alert/environment/hazard/alert-3.0-ascii/consolidated-xml-2.0/20230808000409692/AB_00_01_LAND/STW/2340035092045549953202308070501_AB_00_01_LAND/actual/en_proper_complete_c-fr_proper_complete_c/NinJo</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.1:CAP_count</valueName>
            <value>A:7953 M:43700 C:52914</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Alert_Location_Status</valueName>
            <value>ended</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Alert_Name</valueName>
            <value>alerte d'orages violents</value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.0:Alert_Coverage</valueName>
            <value>Alberta</value>
        </parameter>
        <parameter>
            <valueName>layer:SOREM:1.0:Broadcast_Text</valueName>
            <value>À 18h04 heure avancée des Rocheuses le lundi, Environnement Canada a terminé une alerte d'orages violents pour le nord-ouest  de l'Alberta.  Cleardale et Hines Creek. Les orages violents ont faibli ou ont quitté la région. </value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.1:Designation_Code</valueName>
            <value>AB_00_01_LAND</value>
        </parameter>
        <parameter>
            <valueName>layer:SOREM:2.0:WirelessImmediate</valueName>
            <value>No</value>
        </parameter>
        <parameter>
            <valueName>layer:SOREM:2.0:WirelessText</valueName>
            <value>À 18h04 heure avancée des Rocheuses le lundi, Environnement Canada a terminé une alerte d'orages violents pour cette zone de couverture mobile. </value>
        </parameter>
        <parameter>
            <valueName>layer:EC-MSC-SMC:1.1:Alert_Location_Status</valueName>
            <value>ended</value>
        </parameter>
        <area>
            <areaDesc>cté de Clear Hills près de Cleardale Worsley et Cherry Point</areaDesc>
            <polygon>56.2761,-118.9797 56.2173,-119.0834 56.2298,-119.1816 56.179,-119.3963 56.0766,-119.8228 56.0691,-120.0013 56.1476,-120.0013 56.5097,-120.0139 56.7699,-120.0013 56.7769,-119.4783 56.7699,-118.979 56.3497,-118.9796 56.2761,-118.9797</polygon>
            <geocode>
                <valueName>layer:EC-MSC-SMC:1.0:CLC</valueName>
                <value>078212</value>
            </geocode>
            <geocode>
                <valueName>profile:CAP-CP:Location:0.3</valueName>
                <value>4817062</value>
            </geocode>
        </area>
    </info>
<Signature xmlns="http://www.w3.org/2000/09/xmldsig#" Id="Environment Canada">
<SignedInfo>
<CanonicalizationMethod Algorithm="http://www.w3.org/2001/10/xml-exc-c14n#"/>
<SignatureMethod Algorithm="http://www.w3.org/2001/04/xmldsig-more#rsa-sha256"/>
<Reference URI="">
<Transforms>
<Transform Algorithm="http://www.w3.org/2000/09/xmldsig#enveloped-signature"/>
</Transforms>
<DigestMethod Algorithm="http://www.w3.org/2001/04/xmlenc#sha256"/>
<DigestValue>6v99RGrdxEPJZ/xBPfPdaY0fAeDUmArN4aBLJKiuNFI=</DigestValue>
</Reference>
</SignedInfo>
<SignatureValue>Tr/eBGCKKZykGS1D/srjrx1CVfptgJm66DbfqeXZkE9PIprbT0gt/tA8uSEbHXGT
legxX3fyHKfRsehSwFSZ5E3WJZcsrMe5bZ6Yn+veAj7Y0rfAx8ybzg/UlPaTQEm7
bDCn6SkO2seDSVOsS86b1NlOnIxieWDfpamJTQUHz3EiNM/IlFsWnZoUAyi0vdBi
4suyA8LST1I5171rDahe7It0enfCzaNQhdvDr7gUv+XKdcZXYoweC6zaUtG1pW/R
AxYKLhZ0e4p5qZERwinja4Qm6D6w79xXPgjzzLMXXTXeifDpcGwGOdk9gu0kIFUo
91VI8b/OiUJisKBFx5mssQ==</SignatureValue>
<KeyInfo>
<X509Data>
<X509Certificate>MIIHQDCCBiigAwIBAgIQauIRkpOPcGViI+LoZHWDFzANBgkqhkiG9w0BAQsFADCB
ujELMAkGA1UEBhMCVVMxFjAUBgNVBAoTDUVudHJ1c3QsIEluYy4xKDAmBgNVBAsT
H1NlZSB3d3cuZW50cnVzdC5uZXQvbGVnYWwtdGVybXMxOTA3BgNVBAsTMChjKSAy
MDEyIEVudHJ1c3QsIEluYy4gLSBmb3IgYXV0aG9yaXplZCB1c2Ugb25seTEuMCwG
A1UEAxMlRW50cnVzdCBDZXJ0aWZpY2F0aW9uIEF1dGhvcml0eSAtIEwxSzAeFw0y
MzA0MjcxNDEyNDVaFw0yNDA1MDcxNDEyNDVaMGwxCzAJBgNVBAYTAkNBMQ8wDQYD
VQQIEwZRdWViZWMxETAPBgNVBAcTCEdhdGluZWF1MR8wHQYDVQQKExZTaGFyZWQg
U2VydmljZXMgQ2FuYWRhMRgwFgYDVQQDDA8qLndlYXRoZXIuZ2MuY2EwggEiMA0G
CSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQDOSYFvWlJgQgD5twPwuMqUXQB8Raj6
enUh60KlOqYi2Moja8qBMhANS848XjHd+OlTNKv5J9YT6OoxTqcRe60e3nWYF6/u
Plu+O4KWOO8x6jWbttEn5G7j7V35yaWXqP0LObVrQLDMYX3D4xsm+MJK7eyGJz8k
AJAJYONS/+dp7/0jlrzlTjyuT4Uk50zVDvTGxiYpgjPfuXG9Bke5q8Cju4EEgAfW
1I2+67Is7ae2fTjhvPH+w+L4iujk1ucmkrZqd2Norr8eeLebvDiSNxV4FkdzUZKQ
TTycHQb1u/79WGgRKCH+nPYTzW/l/muRz59V0LKTQOlFmq8H45fmSYWZAgMBAAGj
ggONMIIDiTAMBgNVHRMBAf8EAjAAMB0GA1UdDgQWBBRlq9u/A7i58xA3Eb4DmYis
FzYwKTAfBgNVHSMEGDAWgBSConB03bxTP8971PfNf6dgxgpMvzBoBggrBgEFBQcB
AQRcMFowIwYIKwYBBQUHMAGGF2h0dHA6Ly9vY3NwLmVudHJ1c3QubmV0MDMGCCsG
AQUFBzAChidodHRwOi8vYWlhLmVudHJ1c3QubmV0L2wxay1jaGFpbjI1Ni5jZXIw
MwYDVR0fBCwwKjAooCagJIYiaHR0cDovL2NybC5lbnRydXN0Lm5ldC9sZXZlbDFr
LmNybDCBnQYDVR0RBIGVMIGSgg8qLndlYXRoZXIuZ2MuY2GCDXdlYXRoZXIuZ2Mu
Y2GCC21ldGVvLmdjLmNhgg0qLm1ldGVvLmdjLmNhghUqLmFscGhhLndlYXRoZXIu
Z2MuY2GCEyouYWxwaGEubWV0ZW8uZ2MuY2GCFCouYmV0YS53ZWF0aGVyLmdjLmNh
ghIqLmJldGEubWV0ZW8uZ2MuY2EwDgYDVR0PAQH/BAQDAgWgMB0GA1UdJQQWMBQG
CCsGAQUFBwMBBggrBgEFBQcDAjBMBgNVHSAERTBDMDcGCmCGSAGG+mwKAQUwKTAn
BggrBgEFBQcCARYbaHR0cHM6Ly93d3cuZW50cnVzdC5uZXQvcnBhMAgGBmeBDAEC
AjCCAXsGCisGAQQB1nkCBAIEggFrBIIBZwFlAHUAc9meiRtMlnigIH1HneayxhzQ
UV5xGSqMa4AQesF3crUAAAGHww80IQAABAMARjBEAiAkpowR7bbTYLp4v/tw0Fav
HHP8mm2i6/ecg03yasJkkgIga1hmg5g/Gq97+HOCDwjsy77aS8+PirSarIO6/X1H
EfkAdQA7U3d1Pi25gE6LMFsG/kA7Z9hPw/THvQANLXJv4frUFwAAAYfDDzQcAAAE
AwBGMEQCIBKVCiUTam6W9Y+5TiuhpsxVVx2TYVz9SebYPuB4XyHUAiB4Y9ZXnkOt
rJJAaw4im3kY7OWZG40xPUL1ygK8trUsIAB1AO7N0GTV2xrOxVy3nbTNE6Iyh0Z8
vOzew1FIWUZxH7WbAAABh8MPNF0AAAQDAEYwRAIgYbZkNJlz8a4d35p4+wLg0h5P
WtFyyBEqy06IoSV7UWICIH65mirJoLVsZFK7kXW3IJF4+xlgATBcr/n2ImBLSWug
MA0GCSqGSIb3DQEBCwUAA4IBAQCcp2dfEDsCzJHpMsTF1hlEyYPTgmfrSbgegWT3
76r0r7XGY7qIym53HE6Pk3Q7BgsegpV+pS8UMWx1m9x0ZcUQX11gMBlNLv0WCSBe
4/9BOaGc2deYrXsRAmmFJwqvgegcBSQhg4zZSoSSVqJPhaJ1WYWqDbJW+doY15xy
JPm7UbZqCAbzm3/c36N9j/Zu6h+LA9rhltfwYoWCyqlBtJCRrs7K7KEP+dLtsoby
+AhyY+3dK2IB4G7v5Qbxa88WKV9EsXXMpvl0v/kXSDjIHJwRPyOf1OKU9x2Mc3Mx
KEgzSJ0GUUW0RqqCVWZ14TBoxUq1jJw4m12zSF+PYFpSH6g5</X509Certificate>
</X509Data>
</KeyInfo>
</Signature></alert>"""

In [23]:
root = ET.fromstring(xml_string)

In [24]:
root.tag

'{urn:oasis:names:tc:emergency:cap:1.2}alert'

In [25]:
root.attrib

{}

In [26]:
for child in root:
    print(child.tag, child.attrib)

{urn:oasis:names:tc:emergency:cap:1.2}identifier {}
{urn:oasis:names:tc:emergency:cap:1.2}sender {}
{urn:oasis:names:tc:emergency:cap:1.2}sent {}
{urn:oasis:names:tc:emergency:cap:1.2}status {}
{urn:oasis:names:tc:emergency:cap:1.2}msgType {}
{urn:oasis:names:tc:emergency:cap:1.2}source {}
{urn:oasis:names:tc:emergency:cap:1.2}scope {}
{urn:oasis:names:tc:emergency:cap:1.2}code {}
{urn:oasis:names:tc:emergency:cap:1.2}code {}
{urn:oasis:names:tc:emergency:cap:1.2}code {}
{urn:oasis:names:tc:emergency:cap:1.2}code {}
{urn:oasis:names:tc:emergency:cap:1.2}code {}
{urn:oasis:names:tc:emergency:cap:1.2}note {}
{urn:oasis:names:tc:emergency:cap:1.2}references {}
{urn:oasis:names:tc:emergency:cap:1.2}info {}
{urn:oasis:names:tc:emergency:cap:1.2}info {}
{http://www.w3.org/2000/09/xmldsig#}Signature {'Id': 'Environment Canada'}


In [27]:
ns = {'cap': 'urn:oasis:names:tc:emergency:cap:1.2'}

In [28]:
df_alert = []

In [29]:
for i in root.findall('.//{urn:oasis:names:tc:emergency:cap:1.2}info'):
    language = i.find('{urn:oasis:names:tc:emergency:cap:1.2}language').text
    category = i.find('{urn:oasis:names:tc:emergency:cap:1.2}category').text
    event = i.find('{urn:oasis:names:tc:emergency:cap:1.2}event').text
    responseType = i.find('{urn:oasis:names:tc:emergency:cap:1.2}responseType').text
    urgency = i.find('{urn:oasis:names:tc:emergency:cap:1.2}urgency').text
    severity = i.find('{urn:oasis:names:tc:emergency:cap:1.2}severity').text
    certainty = i.find('{urn:oasis:names:tc:emergency:cap:1.2}certainty').text
    polygon_element = i.find('.//{urn:oasis:names:tc:emergency:cap:1.2}polygon')
    polygon = polygon_element.text if polygon_element is not None else None

In [30]:
df_alert.append([category, event, responseType, urgency, severity, certainty,polygon])

In [31]:
df_alert

[['Met',
  'orages',
  'AllClear',
  'Past',
  'Minor',
  'Observed',
  '56.2761,-118.9797 56.2173,-119.0834 56.2298,-119.1816 56.179,-119.3963 56.0766,-119.8228 56.0691,-120.0013 56.1476,-120.0013 56.5097,-120.0139 56.7699,-120.0013 56.7769,-119.4783 56.7699,-118.979 56.3497,-118.9796 56.2761,-118.9797']]

In [38]:
def process_polygon(polygon_text):
    coordinates = polygon_text.split()

    latitudes = []
    longitudes = []

    for coord in coordinates:
        lat, lon = coord.split(',')
        latitudes.append(lat)
        longitudes.append(lon)

    return ','.join(latitudes), ','.join(longitudes)

df_alert_1['latitudes'], df_alert_1['longitudes'] = zip(*df_alert_1['polygon'].apply(process_polygon))

df_alert_1.drop('polygon', axis=1, inplace=True)

In [37]:
df_alert_1 = pd.DataFrame(df_alert, columns=['category', 'event', 'responseType', 'urgency', 'severity', 'certainty', 'polygon'])

In [40]:
engine.execute("CREATE SCHEMA IF NOT EXISTS p2_weather_data;")
df_alert_1.to_sql(
    name = 'weather_alert',
    schema = 'p2_weather_data',
    con = engine,
    if_exists = 'replace',
    index = False,
    method='multi',
    dtype= {
    'language'            : sa.types.String,
    'category'            : sa.types.String,
    'event'               : sa.types.String,
    'responseType'        : sa.types.String,
    'urgency'             : sa.types.String,
    'severity'            : sa.types.String,
    'certainty'           : sa.types.String,
    'latitudes'           : sa.types.String,
    'longitudes'          : sa.types.String
}
)

### Real-time Recording for Weather Data

In [ ]:
#def extract_and_store_weather_data():
#    appid = '8782e17b1bcc828a0a1ac94b627f05be'
#    lat = 43.6548
#    lon = -79.3883

#    api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}'.format(lat,lon,appid)

#    print(api_url)
    ##ACCESS URL
#    api_response = requests.get(api_url)
    ##GET DATA
#    data = api_response.json()
    ##CREATE DATA FOR STORING
#    weather_data = pd.DataFrame(data['weather'])
    ##ACCESS MY POSTGRES
#    db_secret = {
#        'drivername' : 'postgresql+psycopg2',
#        'host'       : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
#        'port'       : '5432',
#        'username'   : 'hunterf',
#        'password'   : '2023!Schulich',
#        'database'   : 'hunterf_db'
#    }
    #PREPARE ENGINE INFO
#    db_connection_url = sa.engine.URL.create(
#        drivername = db_secret['drivername'],
#        username   = db_secret['username'],
#        password   = db_secret['password'],
#        host       = db_secret['host'],
#        port       = db_secret['port'],
#        database   = db_secret['database']
#    )
    #CREATE ENGINE
#    engine = sa.create_engine(db_connection_url)
    #STORE DATA
#    engine.execute("CREATE SCHEMA IF NOT EXISTS p2_weather_data;")

#    weather_data.to_sql(
#        name = 'weather_data',
#        schema = 'p2_weather_data',
#        con = engine,
#        if_exists = 'replace',
#        index = False,
#        method='multi',
#        dtype= {
#            'id'          : sa.types.DECIMAL(10,0),
#            'main'        : sa.types.String,
#            'description' : sa.types.String,
#            'icon'        : sa.types.String
#    }
#    )

#while True:
#    extract_and_store_weather_data()
#    time.sleep(86400)